# Data description
In order to collect data in a natural way:
<br>- we registered Twitter Developer account;
<br>- using credentials from Twitter Developer account we run script that collected tweets by the geolocation and saved them in mongodb;
<br>
<br><b>As a result:</b>
<br>- we collected  332548 tweets (10Gb in mongodb, ~100Mb in csv) from New-York geolocation since 30 of May up to 15 of June;
<br>- we collected  6617029 tweets (~1.69Gb in csv) from USA geolocation since 15 of June up to now.

### Import all needed libs

In [1]:
# Do not move
import findspark
findspark.init()

In [3]:
import pyspark
import operator
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType, StructField, StructType
from pyspark.sql.functions import udf, row_number,column

# processing
import re
from datetime import datetime

# text preprocessing
import re
import nltk
from nltk.stem import WordNetLemmatizer 
from pyspark.ml.feature import CountVectorizer,StopWordsRemover, HashingTF, IDF, Tokenizer

#staff for LDA
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector as oldVector, Vectors as oldVectors
from pyspark.ml.linalg import Vector as newVector, Vectors as newVectors

# import hardcoded variables
from variables import channels_not_to_consider

#for debug purpose only
import time

#pytrends - for acquiring google trends
from get_google_trends_data.pytrends.pytrends.request import TrendReq

# basically spark

In [4]:
# nltk.download('stopwords')
# nltk.download('wordnet')

### Datetype functions

In [5]:
wrong_date = datetime.strptime("Mon Jun 03 00:00:00 +0000 2000", '%a %b %d %H:%M:%S %z %Y')

def validate(date_text):
    try:
        if date_text != datetime.strptime(date_text, '%a %b %d %H:%M:%S %z %Y').strftime('%a %b %d %H:%M:%S %z %Y'):
            raise ValueError
        return True
    except ValueError:
        return False

def str_tweet_to_datetime(frame_datetime):
    if (validate(frame_datetime) == True):
        return datetime.strptime(frame_datetime,'%a %b %d %H:%M:%S %z %Y')
    else:
        return wrong_date

def datetime_to_tweet_str(frame_datetime):
    #print(type(frame_datetime))
    ts = datetime.strftime(frame_datetime, '%a %b %d %H:%M:%S %z %Y')
    return ts

In [6]:
def get_history_and_real_timeframe(requested_start_dt, requested_finish_dt):
    
    const_end_history_datetime = str_tweet_to_datetime("Fri Jul 05 00:00:00 +0000 2019")

    history_start_datetime = None
    history_finish_datetime = None
    realtime_start_datetime = None
    realtime_finish_datetime = None

    assert requested_finish_dt > requested_start_dt, "Finish dataframe MUST be greater than start"

    if (requested_start_dt >= const_end_history_datetime and requested_finish_dt > const_end_history_datetime):
        realtime_start_datetime = requested_start_dt
        realtime_finish_datetime = requested_finish_dt
    elif (requested_start_dt < const_end_history_datetime and requested_finish_dt <= const_end_history_datetime):
        history_start_datetime = requested_start_dt
        history_finish_datetime = requested_finish_dt
    else:
        history_start_datetime = requested_start_dt
        history_finish_datetime = const_end_history_datetime
        realtime_start_datetime = const_end_history_datetime
        realtime_finish_datetime = requested_finish_dt
        
    return (history_start_datetime, history_finish_datetime, realtime_start_datetime, realtime_finish_datetime)

### Global variables definition

In [7]:
#final of league championship 
lc_final_start_datetime = "Sat Jun 01 00:00:00 +0000 2019"
lc_finish_finish_datetime = "Sat Jun 01 23:59:59 +0000 2019"

#Stanley cup final
stanley_final_start_datetime = "Wed Jun 12 00:00:00 +0000 2019"
stanley_finish_finish_datetime = "Wed Jun 12 23:59:59 +0000 2019"

#Draft NBA
nba_final_start_datetime = "Thu Jun 20 00:00:00 +0000 2019"
nba_finish_finish_datetime = "Sun Jun 23 23:59:59 +0000 2019"

**User-specific variables**  
Please feel free to tweak those variables as you wish. For example, you can set number of last hours to get hottest topics.

In [8]:
# if True locations from locations_to_consider will be used to filter
get_from_location = True

# locations to filter relevant tweets
locations_to_consider = [
                         'Manhattan, NY', 
                         'Brooklyn, NY', 
                         'Queens, NY', 
                         'Bronx, NY', 
                         'Staten Island, NY'
                         'New York, USA'
                        ]

geo = "US-NY" #US for USA

number_of_hours_to_get_topics = 2
num_of_top_interest = 15

# Set window time for interesting
frame_start_datetime = str_tweet_to_datetime(stanley_final_start_datetime)
frame_finish_datetime = str_tweet_to_datetime(stanley_finish_finish_datetime)

assert (frame_finish_datetime - frame_start_datetime).days <= 3, "Date interval should not be bigger than 3 days"

**Technical variables**  
Those variables are needed to connect to db and other technical stuff.

In [9]:
# LDA params
num_of_topics_LDA = 10
max_iterations_LDA = 100
nomber_of_words_to_for_topic = 15  # number of words per topic

# path to CSV
historical_tweets_data = './get-tweets-by-geolocation/data/new_york_training_tweets_15_06.csv'
# historical_tweets_data = './get-tweets-by-geolocation/training_tweets.csv'
# MongoDB table
real_time_tweets_table = "usa_training_tweets_04_07.training_tweets_collection"

### Create spark session

In [10]:
spark = SparkSession.builder.appName("pipeline") \
    .config('spark.mongodb.input.uri', 'mongodb://localhost:27017/'+real_time_tweets_table) \
    .config('spark.mongodb.output.uri', 'mongodb://localhost:27017/'+real_time_tweets_table) \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1') \
    .config('spark.mongodb.input.partitioner', 'MongoPaginateBySizePartitioner') \
    .getOrCreate()
sc = spark.sparkContext

### Handy functions

**Text preprocessing and filtering**

In [11]:
def filter_tweet(tweet, channels_not_to_consider):
    
    if not isinstance(tweet, str):
        is_filtered = True
    elif len(tweet.split(' ')) < 3:
        is_filtered = True
    else: 
        is_filtered = False
        
    return not is_filtered
         
def process_tweet(tweet):
   
    tweet = tweet.lower() # get lowercase
    tweet = re.sub(r'@\w+', '', tweet) # filter words with non-letters at the beginning (mainly for mentions)
    tweet = re.sub(r'http://\S{,280}', '', tweet) # filter http
    tweet = re.sub(r'https://\S{,280}', '', tweet) # filter https
    tweet = re.sub(r'[^A-Za-z]', ' ', tweet) # filter all non-letters
    tweet = re.sub(r'\s{2,}', ' ', tweet) # remove multiply whitespaces
    tweet = re.sub(r'(.)\1{2,}', r'\1', tweet) # remove repeated chars (e.g. "greeeeat" -> "great")
    tweet = tweet.strip() # remove possible whitespaces from both sides of the tweet

    # lemmatize, tokenize and conquer
    processed_tweet = [lemmatizer.lemmatize(token) for token in tokenizer.tokenize(tweet)
                       if token not in stop_word_list]
    
    return processed_tweet

#### Google trends

In [12]:
#TODO: move this function to Handy function block 
def get_google_trends_by_geo(geo):
    if geo == 'US':
        return google_trends_search_topics_us, google_trends_search_queries_us
    elif geo == 'US-NY':
        return google_trends_search_topics_us_ny, google_trends_search_queries_us_ny
    
    return None, None

# How to call this block with functions?

In [13]:
def tweet2google_timeframe(frame_start_datetime, frame_finish_datetime):
    start_date = str_tweet_to_datetime(frame_start_datetime)
    end_date = str_tweet_to_datetime(frame_finish_datetime)
    tim
    
def get_google_trends_by_geo(geo):
    if geo == 'US':
        return google_trends_search_topics_us, google_trends_search_queries_us
    elif geo == 'US-NY':
        return google_trends_search_topics_us_ny, google_trends_search_queries_us_ny
    
    return None, None

In [14]:
#TODO: move this function to utils
def str_rising_to_float(str):
    if str is None:
        return 0.0
    if str == '':
        return 0.0
    if str == 'Breakout':
        return 0.0
    
    str_value = str.split('%')[0]
    if '+' in str_value:
        str_value = str_value.split('+')[1]
        
    if ',' in str_value:
        str_value = str_value.replace(',', '.')
        value = 1000* float(str_value)
        return value
    return float(str_value)

In [15]:
#TODO: move this function to utils
def unique_google_trends_by_time_frame(df):
    data = df.collect()
    rising_dict = {}
    top_dict = {}
    
    geo = data[0]['geo']
    columns = df.columns

    for i in range(0, len(data)):
        rising_val = data[i][columns[1]]
        top_value = data[i][columns[2]]
        
        if rising_val in rising_dict:
            rising_dict[rising_val][0] += str_rising_to_float(data[i][columns[3]])
            rising_dict[rising_val][1] += 1
        else:
            rising_dict[rising_val] = [str_rising_to_float(data[i][columns[3]]), 1]
            
        if top_value in top_dict:
            top_dict[top_value][0] += float(data[i][columns[4]])
            top_dict[top_value][1] += 1
        else:
            top_dict[top_value] = [float(data[i][columns[4]]), 1]
    
    
    for key in top_dict:
        top_dict[key] = round(top_dict[key][0] / top_dict[key][1])
        
    for key in rising_dict:
        rising_dict[key] = round(rising_dict[key][0] / rising_dict[key][1])
    
    top_dict = sorted(top_dict.items(), key=operator.itemgetter(1), reverse=True)
    rising_dict = sorted(rising_dict.items(), key=operator.itemgetter(1), reverse=True)
    
    
    seq = []
    len_top = len(top_dict)
    len_rising = len(rising_dict)
    length = max(len_top, len_rising)
    
    row = Row(columns[1], columns[2], columns[3], columns[4], columns[5])
    
    for i in range(0, length):
        rising = rising_dict[i][0] if i < len_rising else ''
        rising_val = f"+{rising_dict[i][1]}%" if i < len_rising else None
        
        top = top_dict[i][0] if i < len_top else ''
        top_val = top_dict[i][1] if i < len_top else None
        
        seq.append(row(rising, top, rising_val, top_val, geo))
    
    dframe = spark.createDataFrame(seq)
    return dframe

In [16]:
def get_geo_name(geo):
    if geo == "US-NY":
        return "New York"
    elif geo == "US":
        return "United States"
    return ""

def print_google_trend_title(start_date, finish_date, name):
    start_date_str = start_date.strftime("%Y-%m-%d")
    if start_date == finish_date:
        print(f"\nGoogle trends {name} in {get_geo_name(geo)} during {start_date_str}")
    else:
        finish_date_str = finish_date.strftime("%Y-%m-%d")
        print(f"\nGoogle trends {name} in {get_geo_name(geo)} during {start_date_str} - {finish_date_str}")

In [17]:
def convert_datetime_in_interesting_google(df):
    columns = df.columns
    converted_df = df.rdd.map(lambda x : (
                                          x["Date"].strftime("%Y-%m-%d"), 
                                          x[columns[1]], 
                                          x[columns[2]], 
                                          x[columns[3]],
                                          x[columns[4]],
                                          x[columns[5]])).toDF([columns[0], columns[1], columns[2], columns[3], columns[4], columns[5]])
                                                
    return converted_df

# Load the data


## Loading Google Trends data

In [18]:
google_trends_search_queries_us = spark.read.csv('data/google-trends/google-trends-search-queries-US.csv', inferSchema=True, header=True)
google_trends_search_topics_us = spark.read.csv('data/google-trends/google-trends-search-topics-US.csv', inferSchema=True, header=True)
google_trends_search_queries_us_ny = spark.read.csv('data/google-trends/google-trends-search-queries-US-NY.csv', inferSchema=True, header=True)
google_trends_search_topics_us_ny = spark.read.csv('data/google-trends/google-trends-search-topics-US-NY.csv', inferSchema=True, header=True)

## Reading the historical data, it can take a while

In [19]:
times = get_history_and_real_timeframe(requested_start_dt = frame_start_datetime, 
                                       requested_finish_dt = frame_finish_datetime)

print("Range for csv: ", times[0], times[1])
print("Time range for mongodb: ", times[2], times[3])

Range for csv:  2019-06-12 00:00:00+00:00 2019-06-12 23:59:59+00:00
Time range for mongodb:  None None


In [20]:
def get_historical_df(historical_start_time, historical_finish_time):
    print("Range for collected data (history): ", historical_start_time, historical_finish_time)
    
    df = spark.read.csv(historical_tweets_data, inferSchema=True, header=True)
    # remove records with no date
    df = df.na.drop(subset=["created_at"])
    
    # convert string to desired date format
    from datetime import datetime
    from pyspark.sql.functions import col, udf
    from pyspark.sql.types import DateType, TimestampType

    func =  udf (lambda x: str_tweet_to_datetime(x), TimestampType())

    df = df.withColumn('created_at', func(col('created_at')))

    selected_history = df.filter((df.created_at > historical_start_time) & (df.created_at < historical_finish_time))

    return selected_history

# Mongo DB Staff

In [21]:
# df2 = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [22]:
# df2.createOrReplaceTempView("recent_data")
# df2.printSchema()

In [23]:
# recent_data = spark.sql('SELECT text, screen_name, place.location FROM recent_data WHERE lang="en" AND place.country="US"')

In [24]:
# recent_data.show()

## Data filtering and merging

In [25]:
selected_history = None

if times[0] != None and times[1] != None:
    selected_history = get_historical_df(historical_start_time = times[0], historical_finish_time = times[1])

Range for collected data (history):  2019-06-12 00:00:00+00:00 2019-06-12 23:59:59+00:00


In [26]:
selected_history.count()

29538

In [27]:
# select recent data according to user's time request

times = get_history_and_real_timeframe(requested_start_dt = frame_start_datetime, 
                                       requested_finish_dt = frame_finish_datetime)
recent_start_time = times[2]
recent_finish_time = times[3]

print("Range for recent data (mongodb): ", recent_start_time, recent_finish_time)

selected_recent = None

if recent_start_time != None and recent_finish_time != None:
    selected_recent = df.filter((recent_data.created_at > historical_start_time) 
                                & (recent_data.created_at < historical_finish_time))

Range for recent data (mongodb):  None None


In [28]:
# merge together selected_recent and selected_history
selected_df = None

if selected_history != None and selected_recent != None:
    selected_df = selected_history.union(selected_recent)
elif selected_history != None and selected_recent == None:
    selected_df = selected_history
elif selected_history != None and selected_recent == None:
    selected_df = selected_recent

assert selected_df != None, "Something goes wrong with selecting data from recent data/history data"

In [29]:
selected_df.count()

29538

# Tweets preprocessing

Text cleaning is crucial for any text modelling process, especially for topic modelling. In our case it consists from those steps:  
1) Lowercase all words  
2) Filter words with non-letters at the beginning (mainly for mentions, e.g. "@some_user")  
3) Filter http/https  
4) Filter all non-letters (crucial to remove emoji)  
5) Remove multiply whitespaces  
6) Remove repeated chars (e.g. "greeeeat" -> "great")

In [30]:
df = selected_df

In [31]:
tokenizer = nltk.WordPunctTokenizer()
lemmatizer = WordNetLemmatizer()
stop_word_list = nltk.corpus.stopwords.words('english')

In [32]:
# filter nans
df = df.rdd.filter(lambda x: x[0] != None and x[1] != None and x[2] != None and x[4] != None)

# filter out channels not to consider
df = df.filter(lambda x: x[4] not in channels_not_to_consider)

# filter by country
df = df.filter(lambda x: x[1] in 'US')

# filter by precise location
if get_from_location:
    df = df.filter(lambda x: x[2] in locations_to_consider)

# filter tweet itself
df = df.filter(lambda x: filter_tweet(x[0], channels_not_to_consider=channels_not_to_consider))

# process tweet
df = df.map(lambda x: process_tweet(x[0]))

# final preprocesssing
df = df.filter(lambda x: len(x) > 0)

# make dataframes great again
df = df.map(lambda x: [x])

# schema for df
schema = StructType([StructField('tokens', ArrayType(StringType()), True)])
df = df.toDF(schema=schema)

In [33]:
df.show(10)

+--------------------+
|              tokens|
+--------------------+
|[sad, news, noise...|
|[first, thing, ge...|
|               [lit]|
|[ironically, star...|
|   [thank, bro, bro]|
|         [yeah, sir]|
|[indeed, stan, fo...|
|[good, draping, e...|
|[leftover, tuesda...|
|[anyone, want, em...|
+--------------------+
only showing top 10 rows



In [34]:
df.count()

17398

# Topic modeling/Latent Dirichlet allocation(LDA)

In [38]:
# df2.show(10, True)

In [39]:
# df2.printSchema()

In [40]:
print(time.strftime('%m%d%Y %H:%M:%S'))

cv = CountVectorizer(inputCol="tokens", outputCol="raw_features", vocabSize=10000, minDF=2.0)
cvmodel = cv.fit(df)

print(time.strftime('%m%d%Y %H:%M:%S'))

07172019 22:31:29
07172019 22:32:00


In [41]:
print(time.strftime('%m%d%Y %H:%M:%S'))
df = cvmodel.transform(df)
print(time.strftime('%m%d%Y %H:%M:%S'))

07172019 22:32:00
07172019 22:32:00


In [42]:
idf = IDF(inputCol="raw_features", outputCol="tf_idf_features", minDocFreq=2)
idfModel = idf.fit(df)

df = idfModel.transform(df)


In [43]:
df.show(10, True)

+--------------------+--------------------+--------------------+
|              tokens|        raw_features|     tf_idf_features|
+--------------------+--------------------+--------------------+
|[sad, news, noise...|(7583,[287,369,63...|(7583,[287,369,63...|
|[first, thing, ge...|(7583,[2,21,36,37...|(7583,[2,21,36,37...|
|               [lit]|  (7583,[753],[1.0])|(7583,[753],[6.54...|
|[ironically, star...|(7583,[403,862,11...|(7583,[403,862,11...|
|   [thank, bro, bro]|(7583,[18,150],[1...|(7583,[18,150],[3...|
|         [yeah, sir]|(7583,[120,675],[...|(7583,[120,675],[...|
|[indeed, stan, fo...|(7583,[141,259,42...|(7583,[141,259,42...|
|[good, draping, e...|(7583,[14,79,679,...|(7583,[14,79,679,...|
|[leftover, tuesda...|(7583,[154,1093,1...|(7583,[154,1093,1...|
|[anyone, want, em...|(7583,[20,108,706...|(7583,[20,108,706...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [44]:
#df = df.drop("name")
#df.show(10, False)

In [45]:
w = Window().orderBy(column("tokens"))
df = df.withColumn("id", row_number().over(w))

In [46]:
df.show(10, True)

+--------------------+--------------------+--------------------+---+
|              tokens|        raw_features|     tf_idf_features| id|
+--------------------+--------------------+--------------------+---+
|[aapl, strong, da...|(7583,[0,10,17,21...|(7583,[0,10,17,21...|  1|
|[aaron, coming, b...|(7583,[32,148,144...|(7583,[32,148,144...|  2|
|[aaron, got, full...|(7583,[12,30,106,...|(7583,[12,30,106,...|  3|
|[abandoned, novel...|(7583,[16,115,174...|(7583,[16,115,174...|  4|
|[abingdon, market...|(7583,[1,4,24,708...|(7583,[1,4,24,708...|  5|
|[able, convert, s...|(7583,[11,525,657...|(7583,[11,525,657...|  6|
|[able, cop, gener...|(7583,[54,118,222...|(7583,[54,118,222...|  7|
|           [able, w]|(7583,[76,657],[1...|(7583,[76,657],[4...|  8|
| [aboard, tug, boat]|(7583,[3111,4114]...|(7583,[3111,4114]...|  9|
|[abortion, listen...| (7583,[1494],[1.0])|(7583,[1494],[7.1...| 10|
+--------------------+--------------------+--------------------+---+
only showing top 10 rows



In [47]:
rs = df.rdd.map(lambda x: (x[3], oldVectors.fromML(x[2])))

In [48]:
rs_df = rs.toDF()
rs_df.show(10, False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1 |_2                                                                                                                                                                                                                                                                                                 |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |(7583,[0,10,17,21,35,132,161,233,252,620,637,957,1235],[2.971823584815907,3.6199823781610703,3.861534

In [49]:
# Run the LDA Topic Modeler
# Note the time before and after is printed in order to find out how much time it takes to process x number of records

print(time.strftime('%m%d%Y %H:%M:%S'))
lda_model = LDA.train(rs_df['_1', '_2'].rdd.map(list), k=10, maxIterations=100)
print(time.strftime('%m%d%Y %H:%M:%S'))

07172019 22:34:04
07172019 22:35:18


In [50]:
print(time.strftime('%m%d%Y %H:%M:%S'))
topics = lda_model.topicsMatrix()
vocabArray = cvmodel.vocabulary

07172019 22:35:18


In [51]:
wordNumbers = 15

topicIndices = sc.parallelize(lda_model.describeTopics(maxTermsPerTopic = wordNumbers))

def topic_render(topic):  # specify vector id of words to actual words
    terms = topic[0]
    prob = topic[1]
    
    result = []
    for i in range(nomber_of_words_to_for_topic):
        term = str(round(prob[i],3))+"  "+vocabArray[terms[i]]
        result.append(term)
    return result
print(time.strftime('%m%d%Y %H:%M:%S'))

07172019 22:35:20


In [52]:
print(time.strftime('%m%d%Y %H:%M:%S'))
topics_final = topicIndices.map(lambda topic:topic_render(topic)).collect()
print(time.strftime('%m%d%Y %H:%M:%S'))

07172019 22:35:20
07172019 22:35:20


# Topics

In [51]:
# based on the simple vectors(+number of words)

for topic in range(len(topics_final)):
    print ("Topic #" + str(topic+1) + "")
    for term in topics_final[topic]:
        print (term)
    print ('\n')

Topic #1
0.032  u
0.032  love
0.024  thank
0.02  work
0.02  right
0.019  back
0.017  let
0.016  always
0.014  fuck
0.013  also
0.011  big
0.01  looking
0.01  trying
0.009  watch
0.009  lot


Topic #2
0.03  one
0.029  day
0.026  good
0.026  got
0.022  today
0.021  year
0.018  look
0.018  thing
0.016  much
0.015  last
0.015  feel
0.013  guy
0.012  gonna
0.011  night
0.011  week


Topic #3
0.028  see
0.027  like
0.026  lol
0.015  first
0.014  said
0.013  real
0.013  nigga
0.012  job
0.011  wait
0.011  tell
0.011  many
0.01  wanna
0.009  white
0.009  ok
0.008  team


Topic #4
0.048  new
0.036  york
0.031  time
0.025  go
0.022  think
0.015  come
0.014  oh
0.013  city
0.012  give
0.011  as
0.01  ya
0.01  world
0.01  park
0.009  try
0.008  manhattan


Topic #5
0.022  make
0.021  would
0.019  lmao
0.018  even
0.017  life
0.015  happy
0.015  best
0.012  someone
0.012  w
0.01  thought
0.01  mean
0.009  b
0.009  bitch
0.009  hate
0.009  person


Topic #6
0.039  get
0.028  need
0.025  want
0.013  

### Hot topics in the USA from [Google trends](https://trends.google.com/trends/explore?geo=US)

In [52]:
start_date = frame_start_datetime #str_tweet_to_datetime(frame_start_datetime)
finish_date = frame_finish_datetime #str_tweet_to_datetime(frame_finish_datetime)

In [53]:
google_trends_topics, google_trends_queries = get_google_trends_by_geo(geo) 

##### Google trends search queries

In [54]:
interesting_google_topics = google_trends_topics.filter(
    (google_trends_topics.Date >= start_date) & (google_trends_topics.Date <= finish_date))

In [55]:
print_google_trend_title(start_date, finish_date, "Search topics")
interest_google_topics = convert_datetime_in_interesting_google(interesting_google_topics)
interest_google_topics.select("Date","Search topics - rising", "Search topics - top").show(num_of_top_interest, False)


Google trends Search topics in New York during 2019-06-12 - 2019-06-12
+----------+----------------------+----------------------------------------+
|Date      |Search topics - rising|Search topics - top                     |
+----------+----------------------+----------------------------------------+
|2019-06-12|Download - Topic      |New York - City in New York             |
|2019-06-12|null                  |New York - US State                     |
|2019-06-12|null                  |Google Search - Topic                   |
|2019-06-12|null                  |Google - Technology company             |
|2019-06-12|null                  |2019 - Topic                            |
|2019-06-12|null                  |Weather - Topic                         |
|2019-06-12|null                  |YouTube - Video sharing company         |
|2019-06-12|null                  |Facebook, Inc. - Social network company |
|2019-06-12|null                  |Facebook - Social networking service    |
|201

In case when timeframe is more than 1 day, filter correctly this google-trends

In [56]:
# interesing_google_topics_unique= unique_google_trends_by_time_frame(interesting_google_topics)
# print_google_trend_title(start_date, finish_date, "Search topics")
# interesing_google_topics_unique.select("Search topics - rising", "Search topics - top").show(num_of_top_interest, False)

##### Google trends search queries

In [57]:
interesting_google_queries = google_trends_queries.filter(
    (google_trends_queries.Date >= start_date) & (google_trends_queries.Date <= finish_date))

In [58]:
interesing_google_queries_unique= unique_google_trends_by_time_frame(interesting_google_queries)
print_google_trend_title(start_date, finish_date, "Search queries")
interesing_google_queries_unique.show(num_of_top_interest, False)


Google trends Search queries in New York during 2019-06-12 - 2019-06-12
+-----------------------+--------------------+------+---+-----+
|Search queries - rising|Search queries - top|Rising|Top|geo  |
+-----------------------+--------------------+------+---+-----+
|sudanese massacre      |google              |+300% |100|US-NY|
|hong kong              |weather             |+250% |79 |US-NY|
|ny yankees score       |facebook            |+180% |59 |US-NY|
|iready                 |youtube             |+150% |54 |US-NY|
|boston bruins          |amazon              |+150% |49 |US-NY|
|raz kids               |news                |+110% |46 |US-NY|
|bruins                 |definition          |+110% |33 |US-NY|
|cool math games        |craigslist          |+100% |28 |US-NY|
|verizon wireless       |gmail               |+90%  |24 |US-NY|
|bitcoin price          |translate           |+90%  |23 |US-NY|
|scratch                |nba                 |+90%  |23 |US-NY|
|max landis             |instag

In [59]:
# print_google_trend_title(start_date, finish_date, "Search queries")
# interest_google_queries = convert_datetime_in_interesting_google(interesting_google_queries)
# interest_google_queries.select("Date", "Search queries - rising", "Search queries - top").show(num_of_top_interest, False)

#### Hot topics - google trends (directly) (probably this will be removed)

In [60]:
start_date_str = start_date.strftime("%Y-%m-%d")
finish_date_str = finish_date.strftime("%Y-%m-%d")
pytrend = TrendReq()
pytrend.build_payload(kw_list=[' '], geo=geo, timeframe=f"{start_date_str} {finish_date_str}")

##### Search topics

In [61]:
topics_df = pytrend.related_top_search_topics(spark)

In [62]:
print_google_trend_title(start_date, finish_date, "Search topics")
topics_df.select("Search topics - rising", "Search topics - top").show(num_of_top_interest, False)


Google trends Search topics in New York during 2019-06-12 - 2019-06-12
+-------------------------+----------------------------------------+
|Search topics - rising   |Search topics - top                     |
+-------------------------+----------------------------------------+
|Ice - Topic              |New York - City in New York             |
|Mathematical game - Topic|New York - US State                     |
|Design - Topic           |Google - Technology company             |
|nan                      |Google Search - Topic                   |
|nan                      |2019 - Topic                            |
|nan                      |Weather - Topic                         |
|nan                      |YouTube - Video sharing company         |
|nan                      |Facebook - Social networking service    |
|nan                      |Facebook, Inc. - Social network company |
|nan                      |Amazon.com - E-commerce company         |
|nan                      |Defi

##### Search queries

In [63]:
queries_df = pytrend.related_top_search_queries(spark)

In [64]:
print_google_trend_title(start_date, finish_date, "Search queries")
queries_df.show(num_of_top_interest, False)


Google trends Search queries in New York during 2019-06-12 - 2019-06-12
+-----------------------+--------------------+------+---+-----+
|Search queries - rising|Search queries - top|Rising|Top|geo  |
+-----------------------+--------------------+------+---+-----+
|hong kong              |google              |+350% |100|US-NY|
|courtney stodden       |weather             |+300% |80 |US-NY|
|sudanese massacre      |facebook            |+250% |60 |US-NY|
|khloe kardashian       |youtube             |+250% |56 |US-NY|
|cricinfo               |amazon              |+190% |53 |US-NY|
|iready                 |news                |+180% |42 |US-NY|
|neiman marcus          |craigslist          |+110% |27 |US-NY|
|jupiter ed             |world cup           |+100% |25 |US-NY|
|ny lottery post        |instagram           |+90%  |25 |US-NY|
|adidas                 |nba                 |+90%  |25 |US-NY|
|alex morgan            |gmail               |+80%  |22 |US-NY|
|pbs kids               |map   